# Model Trainer
<img src="https://images.ctfassets.net/wt70guc1rpin/wp-media-31995/b7c03abc540764a4735247d34b590d49/aW8ehS-tyXOhnQL5U103N77RXn4R5QwNGgoq2-Ba3L1NN45gbXq0tX7QQAlUf5LCzehQxVoVOzRoKZL5uK3PsT-rmIRfu-aPDree5fgHGJjb0Fpb6TehBv8Zg9hx.png">

In [2]:
# to subpress warning
import warnings
warnings.filterwarnings("ignore")


# The OS module in Python provides functions for interacting with the operating system.
import os

# Matplotlib is a data visualization and graphical plotting library for Python.
import matplotlib.pyplot as plt

# seaborn is alse a data visualization and graphical plotting library for Python.
import seaborn as sn

# Used to display markdown,image,control (frontend utilities)
from IPython.display import display, clear_output

# NumPy is a Python library used for working with arrays
import numpy as np

import pandas as pd

import pickle

# used to split dataset(features and target) into test and test
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

# shuffle the dataset for a even mixture of each type of feature and target.it gives better result.
from sklearn.utils import shuffle

# Computer vision library
import cv2

# Used to manipulate different parts of the Python runtime environment.
import sys

# An open-source framework for building pipelines to perform computer vision inference.
import mediapipe as mp

# Time module provide time-related functions
import time

# Used to generate random numbers
import random

from math import log10, sqrt
# for mathematical operations

# Keras is a library that provides a Python interface for artificial neural networks. 
# Keras acts as an interface for the TensorFlow library.
import keras

# 1. Keras layers are the building blocks of the Keras library that can be stacked together for creating neural network models.
# 2. Keras Conv2D creates a 2D convolution kernel that is wind with layers input which helps produce a tensor of outputs.
# 3. maxpooling2D Downsamples the input along its spatial dimensions by taking the maximum value over an input window for each channel of the input. 
# 4 .Flattening is converting the data into a 1-dimensional array for inputting it to the next layer. 
# 5 .Dropout regularization technique for reducing overfitting in neural networks by preventing complex co-adaptations on training data.
# 6. Batch normalization is a technique for training very deep neural networks that standardizes the inputs to a layer for each mini-batch. 
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization

#  Model groups layers into an object with training and inference features.
from tensorflow.keras.models import Model

import tensorflow as tf

# Optimizers are the extended class, which include added information to train a specific model.  
# The optimizers are used for improving speed and performance for training a specific model.
# Adam is a stochastic gradient descent method based on adaptive estimation of first-order and second-order moments.
from tensorflow.keras.optimizers import Adam

# importing Sqquential model
from tensorflow.keras import Sequential

# tensorflow.keras.callbacks is used to visualize training of a model.
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint 

In [3]:
# GPU Info 
try:
    import GPUtil
except:
    !pip install GPUtil
    import GPUtil

    
# check physical computing devices
device=tf.config.experimental.list_physical_devices()
for i in device:
    print(i)
if len(device)>1:
    # find GPU details
    print("="*20, "GPU Details", "="*20)
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(gpu_id,gpu.name, gpu.driver,gpu.memoryTotal,gpu.temperature)



PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


In [4]:
PATH=r"D:/Image_datasets/asl_dataset"

In [5]:
# Loading data from CSV and preprocessing
def load_CSV(dataset="dataset.csv"):
    import pandas as pd
    df = pd.read_csv(dataset,header=None,index_col=None)
    
    return df
df1=load_CSV()
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,0/hand1_0_bot_seg_1_cropped.jpeg,Right,123,133,136,130,154,117,179,111,...,66,155,24,216,34,248,51,268,64,0
1,0/hand1_0_bot_seg_2_cropped.jpeg,Right,129,107,141,113,164,100,201,92,...,56,155,6,206,13,234,31,253,45,0
2,0/hand1_0_bot_seg_3_cropped.jpeg,Right,125,102,141,92,158,76,189,70,...,69,138,30,186,28,215,42,236,54,0
3,0/hand1_0_bot_seg_4_cropped.jpeg,Right,141,122,150,133,157,136,164,145,...,91,206,12,258,32,280,58,293,78,0
4,0/hand1_0_bot_seg_5_cropped.jpeg,Left,135,152,153,148,174,123,196,101,...,80,160,32,219,45,247,65,265,81,0


In [6]:
unique_hand=df1[1].unique()
unique_sign=df1[44].unique()
print(unique_hand)
print(unique_sign)
print(df1[1].value_counts())
print(df1[44].value_counts())

['Right' 'Left']
['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h'
 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
Left     2448
Right      67
Name: 1, dtype: int64
m    70
w    70
x    70
y    70
g    70
l    70
o    70
i    70
k    70
b    70
5    70
a    70
n    70
v    70
3    70
s    70
7    70
j    70
r    70
h    70
f    70
9    70
u    70
z    70
1    70
p    70
c    70
q    70
0    70
6    70
4    70
2    70
8    70
e    70
d    70
t    65
Name: 44, dtype: int64


In [7]:
df = pd.get_dummies(df1, columns = [1, 44])
df = df.sample(frac = 1)
df.columns

Index([        0,         2,         3,         4,         5,         6,
               7,         8,         9,        10,        11,        12,
              13,        14,        15,        16,        17,        18,
              19,        20,        21,        22,        23,        24,
              25,        26,        27,        28,        29,        30,
              31,        32,        33,        34,        35,        36,
              37,        38,        39,        40,        41,        42,
              43,  '1_Left', '1_Right',    '44_0',    '44_1',    '44_2',
          '44_3',    '44_4',    '44_5',    '44_6',    '44_7',    '44_8',
          '44_9',    '44_a',    '44_b',    '44_c',    '44_d',    '44_e',
          '44_f',    '44_g',    '44_h',    '44_i',    '44_j',    '44_k',
          '44_l',    '44_m',    '44_n',    '44_o',    '44_p',    '44_q',
          '44_r',    '44_s',    '44_t',    '44_u',    '44_v',    '44_w',
          '44_x',    '44_y',    '44_z'],
      dtyp

In [8]:
filenames=df[0]
filenames

484       6/hand5_6_bot_seg_5_cropped.jpeg
718     a/hand1_a_right_seg_4_cropped.jpeg
799       b/hand2_b_bot_seg_5_cropped.jpeg
2376      y/hand1_y_bot_seg_2_cropped.jpeg
1916      r/hand2_r_bot_seg_2_cropped.jpeg
                       ...                
1323      i/hand5_i_bot_seg_4_cropped.jpeg
1279    i/hand1_i_right_seg_5_cropped.jpeg
2132     u/hand2_u_left_seg_3_cropped.jpeg
1097      f/hand2_f_top_seg_3_cropped.jpeg
1588      m/hand2_m_top_seg_4_cropped.jpeg
Name: 0, Length: 2515, dtype: object

In [9]:
cols=list(range(1,45))
features=df[df.columns[cols]]
features

,2,3,4,5,6,7,8,9,10,11,...,36,37,38,39,40,41,42,43,1_Left,1_Right
484,153,349,196,317,220,259,185,202,140,175,...,109,204,115,148,137,159,149,187,1,0
718,177,359,264,304,325,201,323,97,312,28,...,89,192,98,153,121,212,119,226,1,0
799,193,361,243,312,262,235,223,181,172,170,...,139,204,132,153,130,117,129,84,1,0
2376,179,378,253,323,304,219,332,130,371,79,...,75,194,52,126,33,89,15,44,1,0
1916,203,380,249,333,260,263,221,216,174,187,...,136,238,126,202,146,237,162,265,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,164,330,237,297,290,239,285,171,238,145,...,127,157,112,94,117,56,124,17,1,0
1279,182,371,250,321,294,233,270,152,211,126,...,110,178,96,106,92,61,87,16,1,0
2132,206,341,255,294,264,217,223,167,178,162,...,134,221,134,178,159,203,172,230,1,0
1097,176,361,230,317,264,242,280,172,248,134,...,129,204,128,155,124,120,121,87,1,0


In [10]:
cols=["44_"+x for x in unique_sign]
targets=df[cols]
targets

,44_0,44_1,44_2,44_3,44_4,44_5,44_6,44_7,44_8,44_9,...,44_q,44_r,44_s,44_t,44_u,44_v,44_w,44_x,44_y,44_z
484,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
718,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
799,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2376,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1916,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1279,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# the data for training and the remaining 10% for testing
train_x,test_x,train_y,test_y,trainFilenames,testFilenames = train_test_split(features, targets, filenames, test_size=0.10,random_state=42)


In [12]:
print(len(train_x),len(test_x))
print(len(train_y), len(test_y))
print(len(trainFilenames), len(testFilenames))
print(train_x.shape)
print(train_y.shape)

2263 252
2263 252
2263 252
(2263, 44)
(2263, 36)


In [13]:
# KPD23 - Key Point Detection
def kpd23():
    # Create model here
    model = Sequential()
    model.add(Dense(40, input_dim = 44, activation = 'relu')) # Rectified Linear Unit Activation Function
    model.add(Dense(40, activation = 'relu'))
    model.add(Dense(36, activation = 'softmax')) # Softmax for multi-class classification
    # Compile model here
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

    
    

In [14]:
model=kpd23()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                1800      
                                                                 
 dense_1 (Dense)             (None, 40)                1640      
                                                                 
 dense_2 (Dense)             (None, 36)                1476      
                                                                 
Total params: 4,916
Trainable params: 4,916
Non-trainable params: 0
_________________________________________________________________


In [15]:
EPOCH=60
BATCH_SIZE=10
start_time=time.time()

history = model.fit(train_x, train_y, epochs=EPOCH, batch_size = BATCH_SIZE)
training_time=time.time()-start_time

Epoch 1/60
227/227 [==============================] - 4s 6ms/step - loss: 35.7276 - accuracy: 0.1653
Epoch 2/60
227/227 [==============================] - 1s 6ms/step - loss: 3.0455 - accuracy: 0.4985
Epoch 3/60
227/227 [==============================] - 1s 6ms/step - loss: 1.3866 - accuracy: 0.7035
Epoch 4/60
227/227 [==============================] - 1s 6ms/step - loss: 1.1303 - accuracy: 0.7446
Epoch 5/60
227/227 [==============================] - 1s 6ms/step - loss: 0.8053 - accuracy: 0.8029
Epoch 6/60
227/227 [==============================] - 1s 6ms/step - loss: 0.6718 - accuracy: 0.8290
Epoch 7/60
227/227 [==============================] - 1s 6ms/step - loss: 0.6352 - accuracy: 0.8308
Epoch 8/60
227/227 [==============================] - 1s 6ms/step - loss: 0.7800 - accuracy: 0.7972
Epoch 9/60
227/227 [==============================] - 1s 6ms/step - loss: 0.6363 - accuracy: 0.8263
Epoch 10/60
227/227 [==============================] - 1s 6ms/step - loss: 0.5389 - accuracy: 0.857

In [19]:
model.save("./model.h5", save_format="h5")
print("Training Time : {} S".format(round(training_time,2)))

Training Time : 83.47 S


In [17]:
test_object={
    "test_x":test_x,
    "test_y":test_y,
    "testFilenames":testFilenames,
    "unique_sign":unique_sign
}

with open('test_data.pkl', 'wb') as f:
        pickle.dump(test_object, f)